# Classification des images : Lasagnes vs. Hot-Dog (vs. Hamburgers vs. Raviolis)

Dans ce notebook, nous allons nous intéresser à la reconnaissance d'images, en bon anglais : **Computer Vision**

Nous aborderons les éléments suivants : 
* Les réseaux de convolutions (a.k.a. convnets a.k.a. CNN)
* L'augmentation des données 
* La réutilisation et l'adaptation de réseaux existants



## Le MNIST (le Hello World de la classification d'images)

Le MNIST (Mixed National Institute of Standards and Technology) est une base de données contenant 70 000 (60 000 pour l'entrainement et 10 000 pour la validation) chiffres écrits à la main. Avec ce jeu de données, nous pouvons construire des algorithmes permettant de reconnaitre les chiffres.

Un réseau de neurones traditionnel (totalement connecté), comme pour le précédent exercice de classification du vin, fonctionnera dans une certaine mesure. Cependant, les performances (qualité du modèle évaluée avec la précision des résultats) seront bien meilleures avec un réseau de convolution.

### GPU or not GPU?

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### Construction du réseau

Notre réseau prendra en paramètres des images de chiffres écrits manuellement, ces images ayant pour dimension 28 pixels sur 28, et un seul canal colorimétrique (une image en niveau de gris pour le reformuler clairement)

Création du modèle et ajout des 3 premières couches (Convolution) : 

In [ ]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Description du modèle

In [ ]:
model...

Ajout des couches totalement connectées (Classifier)

In [ ]:
model.add(layers.Flatten())
"""
    Ajout d'une couche dense (64 neurones) avec une activation relu
"""
model.add(...)
"""
    Ajout d'une couche dense (10 neurones car 10 catégories) avec une activation (à choisir ;) )
"""
model.add(...)

In [ ]:
model.summary()

### Téléchargement des données

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical
from matplotlib import pyplot as plt

%matplotlib inline

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

### Affichages des 9 premières images

In [ ]:
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(train_images[i], cmap='gray', interpolation='none')
    plt.title("Class {}".format(train_labels[i]))

Format des images

In [ ]:
train_images.shape

### Préparation et redimensionnement des données d'entrée

Avant de procéder à l'entrainement, nous redimensionnons les données pour qu'elles soient compatibles avec le format d'entrée de notre réseau (qui contient 4 dimensions).

Before training, we’ll preprocess the data by reshaping it into the shape the network
expects and scaling it so that all values are in the [0, 1] interval. Previously, our train-
ing images, for instance, were stored in an array of shape (60000, 28, 28) of type
uint8 with values in the [0, 255] interval. We transform it into a float32 array of
shape (60000, 28 * 28) with values between 0 and 1.

In [ ]:
"""
    Nous ajoutons 1 dimension pour 1 canal (le canal des niveaux de gris)
"""
train_images = train_images.reshape( (60000, ... , ... , ...) )

Nous divisons ensuite les valeurs de "pixels" de l'images pour qu'elles soient comprises entre 0 et 1. (au lieu de 0 à 255)

In [ ]:
train_images = train_images.astype('float32') / ...

In [ ]:
train_images.shape

In [ ]:
"""
    Nous effectuons la même manipulation pour les données de test
"""
test_images = test_images.reshape((10000, ... , ... , ...))
test_images = test_images.astype('float32') / ...

"""
    Nous transformons les données cibles en catégories grâce à la fonction to_categorical fournie par keras 
"""
train_labels = ...
test_labels = ...

"""
    Nous pouvons  compiler le modèle
"""

model.compile(
    optimizer='rmsprop',
    loss=...,
    metrics=[...])

"""
    Nous procédons enfin à l'entrainement du modèle, avec 5 epochs et des batchs de taille 64
"""

model.fit(..., ..., ..., ...)

### Evaluation du modèle

In [ ]:
model.evaluate(test_images, test_labels)

## Lasagnes vs Hots-Dogs

Classifier des chiffres manuscrits est un passage obligé lorsque l'on débute dans la classification d'images. Mais ce type d'architecture peut être utilisé pour classifiers tout type d'images.

Ici, nous allons classifier des photos de plats, issus du dataset food101 ( https://www.vision.ee.ethz.ch/datasets_extra/food-101/ )

Dans ce dataset nous avons 100 catégories de plats. Chaque catégorie contenant 1000 images.
1000 images c'est à la fois beaucoup (il a fallu récupérer ces photographies, les classifier manuellement, etc.) mais c'est très peu pour un algorithme de reconnaissance d'images qui a besoin de beaucoup plus d'images.

Cette exercice reprend l'article du blog Keras 'Cats vs. Dogs' (https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html). L'article d'origine permet de classifier entre 2 catégories (classification binaire). Notre objectif lors de cet atelier est de classifier parmi 4 catégories.

### Réutilisation d'un réseau existant

Nous pourrions créer notre propre réseau de convolution et l'entrainer avec les 100 000 images du dataset pour calculer les différents paramètres du modèle. Malheureusement, cela serait un peu long (vous pouvez essayer chez vous...).

Une autre approche serait de se baser sur un réseau existant. Keras propose différents réseaux pré-entrainés et prêt à l'emploi ( https://keras.io/applications/ ): 

* VGG16
* VGG19
* ResNet50
* Inception V3
* Xception
* ..

Ces réseaux ont été entrainés sur le dataset ImageNet : 1.4 millions d'images classifiées en 1000 catégories. Outre le fait que ces réseaux fonctionnent déjà très bien pour la reconnaissance d'images, le fait de les avoir entrainés sur un tel volume d'images leur a permis de comprendre de quoi était composé une image : formes, contours, plans, etc.

Lorsque nous réutilisons (en les adaptant) ces réseaux pré-entrainés, nous bénéficions de cette connaissance ce qui induit un gain de temps non-négligeable.






### Import des librairies

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from IPython.display import Image, display


model = VGG16(weights='imagenet', include_top=True)

In [ ]:
def predict(photo):
    """
        Analyse l'image et recherche les catégories auxquelles elle appartient
    """
    img = image.load_img(photo, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    preds = decode_predictions(features)
    return preds

def display_photo(photo):
    display(Image(filename=photo))
    
def display_and_predict(photo):
    display_photo(photo)
    print("Predictions = " + str(predict(photo)))  

Une lasagne...

In [ ]:
display_and_predict('./snowcamp/datasets/images/train/lasagna/1089702.jpg')

In [ ]:
display_and_predict('snowcamp/datasets/images/train/lasagna/2009224.jpg')

In [ ]:
display_and_predict('snowcamp/datasets/images/train/hot_dog/1000288.jpg')

In [ ]:
display_and_predict('snowcamp/datasets/images/train/hot_dog/302949.jpg')

### Constantes

In [ ]:
train_data_dir = 'snowcamp/datasets/images/train/'
validation_data_dir = 'snowcamp/datasets/images/validation/'
"""
    Les largeurs et hauteurs d'images devraient être 224
"""
img_width, img_height = ... , ...

"""
    Nombre de catégories (2 mini). Il faudra adapter les contenus de <train_data_dir> et <validation_data_dir> en fonction
"""
nb_categories = 2

batch_size = 50

"""
    800 images / catégorie pour l'entrainement
"""
nb_train_samples = nb_categories * 800

"""
    200 images / catégorie pour la validation
"""
nb_validation_samples = nb_categories * 200

### Data augmentation

Avec peu d'images, il est peu probable qu'un réseau de convolution généralise correctement. Il ne saura pas catégoriser de nouvelles images (jamais vues) alors qu'il sera très bon sur les images utilisées lors de l'entrainement (Overfitting).

Pour éviter ce phénomène, nous allons générer de nouvelles images issues de celles en notre possession et auxquelles nous  avons appliquer un certain nombre de transformations aléatoires. 

La classe ImageDataGenerator permet d'effectuer tout cela avec Keras.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

These are just a few of the options available (for more, see the Keras documentation).
Let’s quickly go over this code:
* rotation_range is a value in degrees (0–180), a range within which to randomly rotate pictures.
* width_shift and height_shift are ranges (as a fraction of total width or
height) within which to randomly translate pictures vertically or horizontally.
* shear_range is for randomly applying shearing transformations.
* zoom_range is for randomly zooming inside pictures.
* horizontal_flip is for randomly flipping half the images horizontally—rele-
vant when there are no assumptions of horizontal asymmetry (for example,
real-world pictures).
* fill_mode is the strategy used for filling in newly created pixels, which can
appear after a rotation or a width/height shift.

In [ ]:
"""
    Initalisez le  ImageDataGenerator en positionnant les attributs:
    *  rotation_range
    *  width_shift_range
    * height_shift_range
    * shear_range
    * zoom_range
"""
datagen = ImageDataGenerator(
    ...,
    ...,
    ...,
    ...,
    ...,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
"""
    Sélection d'une image sur laquelle nous allons appliquer nos transformations aléatoires
"""
import os 

from keras.preprocessing import image
fnames = [os.path.join(train_data_dir + 'lasagna/', fname) for
fname in os.listdir(train_data_dir + 'lasagna/')]
img_path = fnames[3]

img = image.load_img(img_path, target_size=(150, 150))

In [ ]:
"""
    Visualisation des transformations
"""
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

In [ ]:
def save_bottleneck_features():

    """
        Nous allons donc générer de nouvelles images, et passer chacune de ces images dans les couches de convolution (
        en enlevant le classifier)
        Nous stockons ensuite les matrices résultantes dans des fichiers (nommés bottleneck_*) que nous réutiliserons par la suite.
    """
    datagen = ...

    """
        Chargement du modèle (en ignorant le classifier)
    """
    model = VGG16(include_top=... , weights='imagenet')

    print ("Create train matrix...")
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    
    print ("Bottleneck features are OK...")
    np.save(open('bottleneck_features_train_all_v2.npy', 'wb'), bottleneck_features_train)

    print ("Create validation matrix...")
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    
    np.save(open('bottleneck_features_validation_all_v2.npy', 'wb'),
            bottleneck_features_validation
    

In [ ]:
save_bottleneck_features()

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
"""
    Nombre d'epochs (50 pour commencer)
"""
epochs = ...
top_model_weights_path='bottleneck_fc_model_all_v2.h5'

#Create labels
def create_target_row(nb_categories, cat):
    res = [0] * nb_categories
    res[cat] = 1
    return res

def create_target(rows_by_cat, nb_cat):
    target = []
    for i in range(0,nb_cat):
        for j in range(0, rows_by_cat):
            target.append(create_target_row(nb_cat, i))
    return np.array(target)

def train_top_model():
    train_data = np.load(open('bottleneck_features_train_all_v2.npy', 'rb'))
    # the features were saved in order, so recreating the labels is easy
    """
        On crée autant de lignes que d'images 'train'
    """
    train_labels = create_target( ... , nb_categories)

    validation_data = np.load(open('bottleneck_features_validation_all_v2.npy', 'rb'))
    
    """
        On crée autant de lignes que d'images 'test'
    """
    validation_labels = create_target( ... , nb_categories)

    """
        Ajout du classifier
        La première couche dense aura 256 neurones et une activation relu
        Le dropout aura pour valeur entre 0 et 0.5 (voire plus pour les joueurs ;) )
        Et enfin la dernière couche servira à déterminer la classe résultante
    """
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(... , activation= ...))
    model.add(Dropout(... ))
    model.add(Dense(..., activation = ...))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    
    model.save_weights(top_model_weights_path)

In [ ]:
train_top_model()

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

"""
    On ouvre la matrice des bottleneck features (train)
"""
train_data = np.load(open('bottleneck_features_train_all_v2.npy', 'rb'))

input_tensor = Input( shape=(img_width,img_height ,3) )
base_model = VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)

"""
    On redéfinit le classifier
"""
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(..., activation=...))
top_model.add(Dropout(...))
top_model.add(Dense(..., activation= ...))

top_model.load_weights(top_model_weights_path)

"""
    On charge le modèle
"""

loaded_model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

In [ ]:
"""
    On effectue quelques prédictions !
"""

img = image.load_img('./snowcamp/datasets/images/validation/lasagna/3355991.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
features = loaded_model.predict(x)

In [ ]:
features

In [ ]:
img = image.load_img('./snowcamp/datasets/images/validation/hot_dog/2889560.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
features = loaded_model.predict(x)

In [ ]:
features